使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第九章第二節 液体集熱式太陽熱利用設備）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
from s11_02_solar_radiation import \
    load_solrad, \
    calc_I_s_d_t, \
    get_Theta_ex


# 第九章　自然エネルギー利用設備 / Chapter 9 Natural energy utilization equipment

## 第二節　液体集熱式太陽熱利用設備 / Section 2 Water based solar system

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．全般 / General

#### 5.1 補正集熱量 / Corrected heat collection

採用する設備が太陽熱温水器の場合： 

If the equipment used is a solar water heater:

$$\large
L_{sun,lss,d,t} = L_{sun,lss,sh,d,t}
$$

採用する設備がソーラーシステムの場合： 

If the equipment used is a solar system:

$$\large
L_{sun,lss,d,t} = L_{sun,lss,ss,d,t}
$$

$L_{sun,lss,d,t}$：日付$d$の時刻$t$における1時間当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/h) / Corrected heat collection amount by liquid-collecting solar utilization equipment per hour at time $ t $ of date $ d $  
$L_{sun,lss,sh,d,t}$：採用する設備が太陽熱温水器の場合の日付$d$の時刻$t$における1時間当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/h) / When the equipment to be adopted is the solar water heater, the corrected heat collection amount by the liquid heat collecting solar utilization equipment per hour at the time $ t $ of the date $ d $  
$L_{sun,lss,ss,d,t}$：採用する設備がソーラーシステムの場合の日付$d$の時刻$t$における1時間当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/h) / If the equipment to be adopted is a solar system, the corrected heat collection amount by the liquid heat collecting solar utilization equipment per hour at the time $ t $ on the date $ d $  

In [ ]:
def calc_L_sun_lss_d_t(region, sol_region, ls_type, A_sp, P_alpha_sp, P_beta_sp, W_tnk_ss, Theta_wtr_d, L_dash_k_d_t,
                       L_dash_s_d_t, L_dash_w_d_t,
                       L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t):
    """1時間当たりの液体集熱式太陽熱利用設備による補正集熱量 (1)

    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :param ls_type: 液体集熱式太陽熱利用設備の種類 (太陽熱温水器,ソーラーシステム)
    :type ls_type: str
    :param A_sp: 太陽熱集熱部の有効集熱面積 (m2)
    :type A_sp: float
    :param P_alpha_sp: 太陽熱集熱部の方位角 (°)
    :type P_alpha_sp: float
    :param P_beta_sp: 太陽熱集熱部の傾斜角 (°)
    :type P_beta_sp: float
    :param W_tnk_ss: タンク容量 (L)
    :type W_tnk_ss: float
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :type Theta_wtr_d: ndarray
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_k_d_t: ndarray
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_s_d_t: ndarray
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_w_d_t: ndarray
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_b1_d_t: ndarray
    :param L_dash_b2_d_t: 1時間当たりの浴槽自動湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_b2_d_t: ndarray
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_ba1_d_t: ndarray
    :return: 1時間当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/h)
    :rtype: ndarray
    """
    if ls_type == '太陽熱温水器':
        # 採用する設備が太陽熱温水器の場合

        # 1日当たりの基準集熱量
        Q_sh_d = calc_Q_sh_d(
            A_sp_sh=A_sp,
            P_alpha_sp_sh=P_alpha_sp,
            P_beta_sp_sh=P_beta_sp,
            region=region,
            sol_region=sol_region
        )

        # 補正集熱量 (1-1)
        L_sun_lss_d_t = get_L_sun_lss_sh_d_t(
            Q_sh_d=Q_sh_d,
            L_dash_k_d_t=L_dash_k_d_t,
            L_dash_s_d_t=L_dash_s_d_t,
            L_dash_w_d_t=L_dash_w_d_t,
            L_dash_b1_d_t=L_dash_b1_d_t,
            L_dash_b2_d_t=L_dash_b2_d_t,
            L_dash_ba1_d_t=L_dash_ba1_d_t,
            region=region,
            sol_region=sol_region
        )

        return L_sun_lss_d_t
    elif ls_type == 'ソーラーシステム':
        # 採用する設備がソーラーシステムの場合

        # 1日当たりの基準集熱量
        Q_ss_d = calc_Q_ss_d(
            A_sp_ss=A_sp,
            P_alpha_sp_ss=P_alpha_sp,
            P_beta_sp_ss=P_beta_sp,
            region=region,
            sol_region=sol_region
        )

        # 補正集熱量 (1-2)
        L_sun_lss_d_t = calc_L_sun_lss_ss_d_t(
            Q_ss_d=Q_ss_d,
            W_tnk_ss=W_tnk_ss,
            Theta_wtr_d=Theta_wtr_d,
            L_dash_k_d_t=L_dash_k_d_t,
            L_dash_s_d_t=L_dash_s_d_t,
            L_dash_w_d_t=L_dash_w_d_t,
            L_dash_b1_d_t=L_dash_b1_d_t,
            L_dash_b2_d_t=L_dash_b2_d_t,
            L_dash_ba1_d_t=L_dash_ba1_d_t
        )

        return L_sun_lss_d_t
    else:
        raise ValueError(ls_type)


#### 5.2 補機の消費電力量 / Power consumption of auxiliary equipment

採用する設備が太陽熱温水器の場合： 

If the equipment used is a solar water heater:

$$\large
E_{E,lss,aux,d,t} = 0
$$

採用する設備がソーラーシステムの場合： 

If the equipment used is a solar system:

$$\large
E_{E,lss,aux,d,t} = E_{E,ss,cp,d,t}
$$

$E_{E,lss,aux,d,t}$：日付$d$の時刻$t$における1時間当たりの補機の消費電力量(kWh/h) / Power consumption of accessory per hour at time $ t $ of date $ d $  
$E_{E,ss,cp,d,t}$：日付$d$の時刻$t$における1時間当たりのソーラーシステムの循環ポンプの消費電力量(kWh/h) / Solar system circulation pump power consumption per hour at time $ t $ on date $ d $ 

In [ ]:
def calc_E_E_lss_aux_d_t(region, sol_region, ls_type, pmp_type=None, P_alpha_sp=None, P_beta_sp=None):
    """1日当たりの補機の消費電力量 (2)

    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :param ls_type: 液体集熱式太陽熱利用設備の種類 (太陽熱温水器,ソーラーシステム)
    :type ls_type: str
    :param pmp_type: ソーラーシステムの循環ポンプの種類 (省消費電力型,上記以外の機種)
    :type pmp_type: str
    :param P_alpha_sp: 太陽熱集熱部の方位角 (°)
    :type P_alpha_sp: float
    :param P_beta_sp: 太陽熱集熱部の傾斜角 (°)
    :type P_beta_sp: float
    :return: 1日当たりの補機の消費電力量 (kWh/d)
    :rtype: ndarray
    """
    if ls_type == '太陽熱温水器':
        # 採用する説委が太陽熱温水器の場合 (2-1)
        E_E_lss_aux_d_t = np.zeros(24 * 365)
        return E_E_lss_aux_d_t
    elif ls_type == 'ソーラーシステム':
        # 採用する設備がソーラーシステムの場合
        # 1日当たりのソーラーシステムの循環ポンプの消費電力量 (2-2)
        E_E_lss_aux_d_t = calc_E_E_ss_cp_d_t(pmp_type, P_alpha_sp, P_beta_sp, region, sol_region)
        return E_E_lss_aux_d_t
    else:
        raise ValueError(ls_type)


### 6．太陽熱温水器 / Solar water heater

#### 6.1 補正集熱量 / Corrected heat collection

$L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} = 0$ の場合：

In the case of $L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} = 0$

$$\large
L_{sun,lss,sh,d,t} = 0
$$

$L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} > 0$ の場合：

In the case of $L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} > 0$

$$\large
L_{sun,lss,sh,d,t} = L_{sun,lss,sh,d} \times \frac{L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t}}{L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d}}
$$

$L_{sun,lss,sh,d}$ : 日付$d$における1日当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/d) / Corrected heat collection by liquid collection solar utilization equipment per day on date $ d $  
$L_{sun,lss,sh,d,t}$ : 日付$d$の時刻$t$における1時間当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/h) / Corrected heat collection amount by liquid-collecting solar utilization equipment per hour at time $ t $ of date $ d $  
$L'_{k,d}$ : 日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load on kitchen faucet per day on date $ d $  
$L'_{s,d}$ : 日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in one day bathroom shower faucet on date $ d $  
$L'_{w,d}$ : 日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in the daily wash water faucet at date $ d $  
$L'_{b1,d}$ : 日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub faucet hot water at date $ d $  
$L'_{b2,d}$ : 日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub automatic hot water heating at date $ d $  
$L'_{ba1,d}$ : 日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at the time of the day $ d $ in the bathtub faucet first time  
$L'_{k,d,t}$ : 日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at kitchen faucet per hour at time $ t $ on date $ d $  
$L'_{s,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load in bathroom shower faucet per hour at time $ t $ of date $ d $  
$L'_{w,d,t}$ : 日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load in the faucet per hour at time $ t $ of date $ d $  
$L'_{b1,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at the time of bathtub $ 1 $ per hour at date $ t $ of date $ d $  
$L'_{b2,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at the time of bathtub automatic water heating per hour at time $ t $ of date $ d $  
$L'_{ba1,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at the time of bathtub faucet hot water per hour at time $ t $ of date $ d $  

In [ ]:
def get_L_sun_lss_sh_d_t(Q_sh_d, L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t,
                         region,
                         sol_region):
    """1時間当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/h) (3)

    :param Q_sh_d: 1日当たりの基準集熱量 (MJ/h)
    :type Q_sh_d: ndarray
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_k_d_t: ndarray
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_s_d_t: ndarray
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_w_d_t: ndarray
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_b1_d_t: ndarray
    :param L_dash_b2_d_t: 1時間当たりの浴槽自動湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_b2_d_t: ndarray
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_ba1_d_t: ndarray
    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 1時間当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/h)
    :rtype: ndarray
    """
    # 24時間化
    L_dash_k_d = np.sum(L_dash_k_d_t.reshape((365, 24)), axis=1)
    L_dash_s_d = np.sum(L_dash_s_d_t.reshape((365, 24)), axis=1)
    L_dash_w_d = np.sum(L_dash_w_d_t.reshape((365, 24)), axis=1)
    L_dash_b1_d = np.sum(L_dash_b1_d_t.reshape((365, 24)), axis=1)
    L_dash_b2_d = np.sum(L_dash_b2_d_t.reshape((365, 24)), axis=1)
    L_dash_ba1_d = np.sum(L_dash_ba1_d_t.reshape((365, 24)), axis=1)

    # 1日当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/d) (4)
    L_sun_lss_ss_d = calc_L_sun_lss_sh_d(Q_sh_d, L_dash_k_d, L_dash_s_d, L_dash_w_d, L_dash_b1_d, L_dash_b2_d,
                                         L_dash_ba1_d, region,
                                         sol_region)

    # 24時間化
    L_dash_k_d = np.repeat(L_dash_k_d, 24)
    L_dash_s_d = np.repeat(L_dash_s_d, 24)
    L_dash_w_d = np.repeat(L_dash_w_d, 24)
    L_dash_b1_d = np.repeat(L_dash_b1_d, 24)
    L_dash_b2_d = np.repeat(L_dash_b2_d, 24)
    L_dash_ba1_d = np.repeat(L_dash_ba1_d, 24)

    L_sun_lss_ss_d = np.repeat(L_sun_lss_ss_d, 24)

    L_sun_lss_ss_d_t = np.zeros(24 * 365)

    # (9-1) 節湯補正給湯熱負荷が0の場合
    f1 = (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) == 0
    L_sun_lss_ss_d_t[f1] = 0

    # (9-2) 節湯補正給湯熱負荷 > 0の場合
    f2 = (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) > 0
    L_sun_lss_ss_d_t[f2] = (L_sun_lss_ss_d[f2]
                            * (L_dash_k_d_t[f2] + L_dash_s_d_t[f2] + L_dash_w_d_t[f2] + L_dash_b1_d_t[f2] +
                               L_dash_b2_d_t[f2] + L_dash_ba1_d_t[f2])
                            / (L_dash_k_d[f2] + L_dash_s_d[f2] + L_dash_w_d[f2] + L_dash_b1_d[f2] + L_dash_b2_d[f2] +
                               L_dash_ba1_d[f2]))

    return L_sun_lss_ss_d_t


$\theta_{ex,lwlim,sh} \leq \theta_{ex,s,prd,Ave,d}$の場合： 

In the case of $\theta_{ex,lwlim,sh} \leq \theta_{ex,s,prd,Ave,d}$

$$\large
L_{sun,lss,sh,d} = min(Q_{sh,d} ,  (L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d}) \times f_{sr,uplim,sh})
$$

$\theta_{ex,s,prd,Ave,d} < \theta_{ex,lwlim,sh}$の場合： 

In the case of $\theta_{ex,s,prd,Ave,d} < \theta_{ex,lwlim,sh}$

$$\large
L_{sun,lss,sh,d} = 0
$$

$f_{sr,uplim,sh}$：液体集熱式太陽熱利用設備の分担率上限値(-) / Upper limit value of share rate of liquid thermal collection type solar utilization equipment  
$L_{sun,lss,sh,d}$：日付$d$における1日当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/d) / Corrected heat collection by liquid collection solar utilization equipment per day on date $ d $  
$L'_{k,d}$：日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load on kitchen faucet per day on date $ d $  
$L'_{s,d}$：日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in one day bathroom shower faucet on date $ d $  
$L'_{w,d}$：日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in the daily wash water faucet at date $ d $  
$L'_{b1,d}$：日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub faucet hot water at date $ d $  
$L'_{b2,d}$：日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub automatic hot water heating at date $ d $  
$L'_{ba1,d}$：日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at the time of the day $ d $ in the bathtub faucet first time  
$Q_{sh,d}$：日付$d$における1日当たりの基準集熱量(MJ/d) / Standard heat collection per day for date $ d $  
$\theta_{ex,s,prd,Ave,d}$：日付$d$を基準とした期間平均太陽熱外気温度(℃) / Average solar radiation outside temperature period based on date $ d $  
$\theta_{ex,lwlim,sh}$：液体集熱式太陽熱利用設備を使用できる外気温度の下限(℃) / Lower limit of outside air temperature that can use liquid collection type solar thermal facility  


In [ ]:
def calc_L_sun_lss_sh_d(Q_sh_d, L_dash_k_d, L_dash_s_d, L_dash_w_d, L_dash_b1_d, L_dash_b2_d, L_dash_ba1_d, region,
                        sol_region):
    """1日当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/d) (4)

    :param Q_sh_d: 1日当たりの基準集熱量 (MJ/d)
    :type Q_sh_d: ndarray
    :param L_dash_k_d: 台所水栓における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_k_d: ndarray
    :param L_dash_s_d: 浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_s_d: ndarray
    :param L_dash_w_d: 洗面水栓における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_w_d: ndarray
    :param L_dash_b1_d: 浴槽水栓湯はりにおける節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_b1_d: ndarray
    :param L_dash_b2_d: 浴槽自動湯はりにおける節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_b2_d: ndarray
    :param L_dash_ba1_d: 浴槽水栓さし湯における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_ba1_d: ndarray
    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 1日当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/d)
    :rtype: ndarray
    """
    # 期間平均太陽熱外気温度
    Theta_ex_s_prd_Ave_d = calc_Theta_ex_s_prd_Ave_d(region, sol_region)

    # 液体集熱式太陽熱利用設備を使用できる外気温度の下限
    Theta_ex_lwlim_sh = get_Theta_ex_lwlim_sh()

    # 液体集熱式太陽熱利用設備の分担率の上限値
    f_sr_uplim_sh = get_f_sr_uplim_sh()

    # Theta_ex_lwlim <= Theta_ex_s_prd_Ave_d の場合 (3-1)
    # ひとまず、条件を無視してすべての日について計算する
    L_sun_lss_sh_d = np.min(
        [Q_sh_d, (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) * f_sr_uplim_sh],
        axis=0)

    # Theta_ex_s_prd_Ave_d < Theta_ex_lwlim_sh の場合 (3-2)
    # この条件を満たす日だけ0で上書きする
    L_sun_lss_sh_d[Theta_ex_s_prd_Ave_d < Theta_ex_lwlim_sh] = 0.0

    return L_sun_lss_sh_d


液体集熱式太陽熱利用設備を使用できる外気温度の下限$\theta_{ex,lwlim,sh}$は、5℃とする。液体集熱式太陽熱利用設備の分担率上限値$f_{sr,uplim,sh}$は、0.9とする。 

The lower limit $ \ theta_ {ex, lwlim, sh} $ of the outside air temperature at which the liquid thermal solar energy utilization equipment can be used is 5 °C. The share rate upper limit value $f_{sr, uplim, sh} $ of the liquid heat collecting solar use facility is 0.9.

In [ ]:
def get_Theta_ex_lwlim_sh():
    """液体集熱式太陽熱利用設備を使用できる外気温度の下限

    :return: 液体集熱式太陽熱利用設備を使用できる外気温度の下限
    :rtype: float
    """
    return 5.0


def get_f_sr_uplim_sh():
    """ 液体集熱式太陽熱利用設備の分担率上限値

    :return: 液体集熱式太陽熱利用設備の分担率上限値
    :rtype: float
    """
    return 0.9


#### 6.2 基準集熱量 / Reference heat collection

$$\large
Q_{sh,d} = Q_{sp,sh,d} \times A_{sp,sh} \times f_{sp,sh} \times f_{s,sh}
$$

$A_{sp,sh}$：太陽熱集熱部の有効集熱面積(m<sup>2</sup>) / Effective heat collection area of solar heat collector  
$Q_{sh,d}$：日付$d$における1日当たりの基準集熱量(MJ/d) / Standard heat collection per day for date $ d $  
$Q_{sp,sh,d}$：日付$d$における1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量(MJ/m<sup>2</sup>・d) / The amount of solar radiation per unit area of the solar thermal collector installation surface per day for the date $ d $  
$f_{s,sh}$：液体集熱式太陽熱利用設備のシステム効率(-) / System efficiency of liquid collection solar system  
$f_{sp,sh}$：太陽熱集熱部の集熱効率(-) / Heat collection efficiency of solar heat collector  

In [ ]:
def calc_Q_sh_d(A_sp_sh, P_alpha_sp_sh, P_beta_sp_sh, region, sol_region):
    """1日当たりの基準集熱量 (5)

    :param A_sp_sh: 太陽熱集熱部の有効集熱面積 (m2)
    :type A_sp_sh: float
    :param P_alpha_sp_sh: 太陽熱集熱部設置面の方位角
    :type P_alpha_sp_sh: float
    :param P_beta_sp_sh: 太陽熱集熱部設置面の傾斜角
    :type P_beta_sp_sh: float
    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 1日当たりの基準集熱量 (MJ/d)
    :rtype: ndarray
    """
    # 1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量
    Q_sp_sh_d = calc_Q_sp_sh_d(P_alpha_sp_sh, P_beta_sp_sh, region, sol_region)

    # 太陽熱集熱部の集熱効率
    f_sp_sh = get_f_sp_sh()

    # 液体集熱式太陽熱利用設備のシステム効率
    f_s_sh = get_f_s_sh()

    # 1日当たりの基準集熱量
    Q_sh_d = Q_sp_sh_d * A_sp_sh * f_sp_sh * f_s_sh

    return Q_sh_d


太陽熱集熱部の集熱効率$f_{sp,sh}$は、0.4とする。液体集熱式太陽熱利用設備のシステム効率$f_{s,sh}$は、0.85とする。 

The heat collection efficiency $ f_ {sp, sh} $ of the solar heat collector is 0.4. The system efficiency $ f_ {s, sh} $ of the liquid collection solar system is 0.85.

In [ ]:
def get_f_sp_sh():
    """太陽熱集熱部の集熱効率

    :return:太陽熱集熱部の集熱効率
    :rtype: float
    """
    return 0.4


def get_f_s_sh():
    """液体集熱式太陽熱利用設備のシステム効率

    :return: 液体集熱式太陽熱利用設備のシステム効率
    :rtype: float
    """
    return 0.85


太陽熱集熱部の有効集熱面積$A_{sp,sh}$は、ISO 9488 の 8.6 節「aperture area」に規定される方法により算出した値とするか、IS A 4111 に規定される集熱部総面積又は集熱貯湯部総面積に 0.85 を乗じた値とする。 

The effective heat collection area $ A_ {sp, sh} $ of the solar heat collector is a value calculated by the method specified in Section 8.6 “aperture area” of ISO 9488, or the heat collection specified in IS A 4111. The total area of the part or the total area of the heat collection and storage section is multiplied by 0.85.

In [ ]:
def get_A_sp_sh_JIS_A_4111(A):
    """太陽熱集熱部の有効集熱面積(IS A 4111)

    :param A: 太陽熱集熱部総面積
    :type A: float
    :return: 太陽熱集熱部の有効集熱面積
    :rtype: float
    """
    return A * 0.85


$$\large
Q_{sp,sh,d} = \sum_{t = 0}^{23} ( I_{s,sp,sh,d,t} \times 3600 \times 10^{-6} )
$$

$I_{s,sp,sh,d,t}$：日付$d$の時刻$t$における太陽熱集熱部設置面の単位面積当たりの平均日射量(W/m<sup>2</sup>) / Average solar radiation amount per unit area of solar heat collector installation surface at time $ t $ of date $ d $  
$Q_{sp,sh,d}$：日付$d$における1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量(MJ/(m<sup>2</sup>・d)) / The amount of solar radiation per unit area of the solar thermal collector installation surface per day for the date $ d $  

In [ ]:
def calc_Q_sp_sh_d(P_alpha_sp_sh, P_beta_sp_sh, region, sol_region):
    """1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量 (6)

    :param P_alpha_sp_sh: 太陽熱集熱部設置面の方位角
    :type P_alpha_sp_sh: float
    :param P_beta_sp_sh: 太陽熱集熱部設置面の傾斜角
    :type P_beta_sp_sh: float
    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量 (MJ/(m2・d))
    :rtype: ndarray
    """
    # 太陽熱集熱部設置面の単位面積当たりの平均日射量
    solrad = load_solrad(region, sol_region)
    I_s_sp_sh_d_t = calc_I_s_d_t(P_alpha_sp_sh, P_beta_sp_sh, solrad)

    # 1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量
    tmp = I_s_sp_sh_d_t.reshape(365, 24)
    Q_sp_sh_d = np.sum(tmp, axis=1) * 3600 * 10 ** (-6)

    return Q_sp_sh_d


#### 6.3 期間平均太陽熱外気温度 / Period average solar outside air temperature

$$\large
\theta_{ex,s,prd,Ave,d} = \sum_{i = 0}^{30} \theta_{ex,s,Ave,d - 15 + i} \big/ 31
$$

$\theta_{ex,s,prd,Ave,d}$：日付$d$を基準とした期間平均太陽熱外気温度(℃) / Average solar radiation outside temperature period based on date $ d $  
$\theta_{ex,s,Ave,d}$：日付$d$における日平均太陽熱外気温度(℃) / Daily average solar outside temperature on date $ d $  

In [ ]:
def calc_Theta_ex_s_prd_Ave_d(region, sol_region):
    """期間平均太陽熱外気温度 (7)

    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 期間平均太陽熱外気温度 (℃)
    :rtype: ndarray
    """
    # 日平均太陽熱外気温度
    Theta_ex_s_Ave_d = calc_Theta_ex_s_Ave_d(region, sol_region)

    # 31日平均なので、データを前後15日ずつ拡張する
    tmp_ex = np.zeros(365 + 30)
    tmp_ex[:15] = Theta_ex_s_Ave_d[-15:]
    tmp_ex[15:15 + 365] = Theta_ex_s_Ave_d[:]
    tmp_ex[-15:] = Theta_ex_s_Ave_d[:15]

    # 31日平均
    # 基準日と前後15日の31日に1/31を乗算して合算する。
    tmp = np.convolve(tmp_ex, np.ones(31) / 31, mode='valid')

    return tmp


$$\large
\theta_{ex,s,Ave,d} = \sum_{t = 0}^{23} \theta_{ex,s,d,t} \big/ 24
$$

$\theta_{ex,s,Ave,d}$：日付$d$における日平均太陽熱外気温度(℃) / Daily average solar outside temperature on date $ d $  
$\theta_{ex,s,d,t}$：日付$d$の時刻$t$における太陽熱外気温度(℃) / The solar open air temperature at time $ t $ of date $ d $  

In [ ]:
def calc_Theta_ex_s_Ave_d(region, sol_region):
    """日平均太陽熱外気温度 (8)

    :param region: 省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 日平均太陽熱外気温度 (℃)
    :rtype: ndarray
    """

    # 太陽熱外気温度
    solrad = load_solrad(region, sol_region)
    Theta_ex_s_d = get_Theta_ex(solrad)

    tmp = Theta_ex_s_d.reshape(365, 24)
    tmp = np.sum(tmp, axis=1)

    Theta_ex_s_Ave_d = tmp / 24

    return Theta_ex_s_Ave_d


### 7．ソーラーシステム / Solar system

#### 7.1 補正集熱量 / Corrected heat collection

$L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} = 0$ の場合：

In the case of $L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} = 0$

$$\large
L_{sun,lss,ss,d,t} = 0
$$

$L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} > 0$ の場合：

In the case of $L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} > 0$

$$\large
L_{sun,lss,ss,d,t} = L_{sun,lss,ss,d} \times \frac{L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t}}{L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d}}
$$

$L_{sun,lss,ss,d}$ : 日付$d$における1日当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/d) / Corrected heat collection by liquid collection solar utilization equipment per day on date $ d $  
$L_{sun,lss,ss,d,t}$ : 日付$d$の時刻$t$における1時間当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/h) / Corrected heat collection amount by liquid-collecting solar utilization equipment per hour at time $ t $ of date $ d $  
$L'_{k,d}$ : 日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load on kitchen faucet per day on date $ d $  
$L'_{s,d}$ : 日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in one day bathroom shower faucet on date $ d $  
$L'_{w,d}$ : 日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in the daily wash water faucet at date $ d $  
$L'_{b1,d}$ : 日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub faucet hot water at date $ d $  
$L'_{b2,d}$ : 日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub automatic hot water heating at date $ d $  
$L'_{ba1,d}$ : 日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at the time of the day $ d $ in the bathtub faucet first time  
$L'_{k,d,t}$ : 日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at kitchen faucet per hour at time $ t $ on date $ d $  
$L'_{s,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load in bathroom shower faucet per hour at time $ t $ of date $ d $  
$L'_{w,d,t}$ : 日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load in the faucet per hour at time $ t $ of date $ d $  
$L'_{b1,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at the time of bathtub per hour at time $ t $ of date $ d $  
$L'_{b2,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at the time of bathtub automatic water heating per hour at time $ t $ of date $ d $  
$L'_{ba1,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/h) / Hot water correction hot water supply heat load at the time of bathtub faucet hot water per hour at time $ t $ of date $ d $  


In [ ]:
def calc_L_sun_lss_ss_d_t(Q_ss_d, W_tnk_ss, Theta_wtr_d, L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t,
                          L_dash_b2_d_t,
                          L_dash_ba1_d_t):
    """1時間当たりの液体集熱式太陽熱利用設備による補正集熱量 (9)

    :param Q_ss_d: 1日当たりの基準集熱量 (MJ/d)
    :type Q_ss_d: ndarray
    :param W_tnk_ss: ソーラーシステムのタンク容量 (L)
    :type W_tnk_ss: float
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :type Theta_wtr_d: ndarray
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_k_d_t: ndarrayL
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_s_d_t: ndarray
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_w_d_t: ndarray
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_b1_d_t: ndarray
    :param L_dash_b2_d_t: 1時間当たりの浴槽自動湯はりにおける節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_b2_d_t: ndarray
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯における節湯補正給湯熱負荷 (MJ/h)
    :type L_dash_ba1_d_t: ndarray
    :return: 1時間当たりの液体集熱式太陽熱利用設備による補正集熱量
    :rtype: ndarray
    """
    # 24時間化
    L_dash_k_d = np.sum(L_dash_k_d_t.reshape((365, 24)), axis=1)
    L_dash_s_d = np.sum(L_dash_s_d_t.reshape((365, 24)), axis=1)
    L_dash_w_d = np.sum(L_dash_w_d_t.reshape((365, 24)), axis=1)
    L_dash_b1_d = np.sum(L_dash_b1_d_t.reshape((365, 24)), axis=1)
    L_dash_b2_d = np.sum(L_dash_b2_d_t.reshape((365, 24)), axis=1)
    L_dash_ba1_d = np.sum(L_dash_ba1_d_t.reshape((365, 24)), axis=1)

    # 1日当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/h) (10)
    L_sun_lss_ss_d = calc_L_sun_lss_ss_d(Q_ss_d, W_tnk_ss, Theta_wtr_d, L_dash_k_d, L_dash_s_d, L_dash_w_d, L_dash_b1_d,
                                         L_dash_b2_d, L_dash_ba1_d)
    L_sun_lss_ss_d = np.repeat(L_sun_lss_ss_d, 24)

    L_sun_lss_ss_d_t = np.zeros(24 * 365)

    # 24時間化
    L_dash_k_d = np.repeat(L_dash_k_d, 24)
    L_dash_s_d = np.repeat(L_dash_s_d, 24)
    L_dash_w_d = np.repeat(L_dash_w_d, 24)
    L_dash_b1_d = np.repeat(L_dash_b1_d, 24)
    L_dash_b2_d = np.repeat(L_dash_b2_d, 24)
    L_dash_ba1_d = np.repeat(L_dash_ba1_d, 24)

    # (9-1) 節湯補正給湯熱負荷が0の場合
    f1 = (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) == 0
    L_sun_lss_ss_d_t[f1] = 0

    # (9-2) 節湯補正給湯熱負荷 > 0の場合
    f2 = (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) > 0
    L_sun_lss_ss_d_t[f2] = (L_sun_lss_ss_d[f2]
                            * (L_dash_k_d_t[f2] + L_dash_s_d_t[f2] + L_dash_w_d_t[f2] + L_dash_b1_d_t[f2] +
                               L_dash_b2_d_t[f2] + L_dash_ba1_d_t[f2])
                            / (L_dash_k_d[f2] + L_dash_s_d[f2] + L_dash_w_d[f2] + L_dash_b1_d[f2] + L_dash_b2_d[f2] +
                               L_dash_ba1_d[f2]))

    return L_sun_lss_ss_d_t


$$\large
L_{sun,lss,sh,d} = min(L_{tnk,ss,d} ,  (L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d}) \times f_{sr,uplim,ss})
$$

$f_{sr,uplim,ss}$：液体集熱式太陽熱利用設備の分担率上限値(-) / Upper limit value of share rate of liquid thermal collection type solar utilization equipment  
$L_{sun,lss,ss,d}$：日付$d$における1日当たりの液体集熱式太陽熱利用設備による補正集熱量(MJ/d) / Corrected heat collection by liquid collection solar utilization equipment per day on date $ d $  
$L_{tnk,ss,d}$：日付$d$における1日当たりのソーラーシステムのタンク蓄熱量の上限による補正集熱量(MJ/d) / Corrected heat collection by the upper limit of the tank thermal storage capacity of the solar system per day at date $ d $  
$L'_{k,d}$：日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load on kitchen faucet per day on date $ d $  
$L'_{s,d}$：日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in one day bathroom shower faucet on date $ d $  
$L'_{w,d}$：日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load in the daily wash water faucet at date $ d $  
$L'_{b1,d}$：日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub faucet hot water at date $ d $  
$L'_{b2,d}$：日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at one day bathtub automatic hot water heating at date $ d $  
$L'_{ba1,d}$：日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d) / Hot water correction hot water supply heat load at the time of the day $ d $ in the bathtub faucet first time  


In [ ]:
def calc_L_sun_lss_ss_d(Q_ss_d, W_tnk_ss, Theta_wtr_d, L_dash_k_d, L_dash_s_d, L_dash_w_d, L_dash_b1_d, L_dash_b2_d,
                        L_dash_ba1_d):
    """1日当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/h) (10)

    :param Q_ss_d: 1日当たりの基準集熱量 (MJ/d)
    :type Q_ss_d: ndarray
    :param W_tnk_ss: ソーラーシステムのタンク容量 (L)
    :type W_tnk_ss: ndarray
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :type Theta_wtr_d: ndarray
    :param L_dash_k_d: 1日当たりの台所水栓における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_k_d: ndarray
    :param L_dash_s_d: 1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_s_d: ndarray
    :param L_dash_w_d: 1日当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_w_d: ndarray
    :param L_dash_b1_d: 1日当たりの浴槽水栓湯はりにおける節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_b1_d: ndarray
    :param L_dash_b2_d: 1日当たりの浴槽自動湯はりにおける節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_b2_d: ndarray
    :param L_dash_ba1_d: 1日当たりの浴槽水栓さし湯における節湯補正給湯熱負荷 (MJ/d)
    :type L_dash_ba1_d: ndarray
    :return: 1日当たりの液体集熱式太陽熱利用設備による補正集熱量 (MJ/d)
    :rtype: ndarray
    """
    # 1日当たりのソーラーシステムのタンク蓄熱量の上限による補正集熱量
    L_tnk_ss_d = calc_L_tnk_ss_d(Q_ss_d, W_tnk_ss, Theta_wtr_d)

    # 液体集熱式太陽熱利用設備の分担率上限値
    f_sr_uplim_ss = get_f_sr_uplim_ss()

    # 1日当たりの液体集熱式太陽熱利用設備による補正集熱量 (8)
    L_sun_lss_ss_d = np.min(
        [L_tnk_ss_d, (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) * f_sr_uplim_ss],
        axis=0)

    return L_sun_lss_ss_d


液体集熱式太陽熱利用設備の分担率上限値$f_{sr,uplim,ss}$は、0.9とする。 

The upper limit value $ f_ {sr, uplim, ss} $ of the liquid collection solar energy sharing system is 0.9.

In [ ]:
def get_f_sr_uplim_ss():
    """液体集熱式太陽熱利用設備の分担率上限値

    :return: 液体集熱式太陽熱利用設備の分担率上限値
    :rtype: float
    """
    return 0.9


$$\large
L_{tnk,ss,d} = min(Q_{ss,d} ,  HC_{tnk,ss,d} \times \alpha_{tnk,ss,d})
$$

$L_{tnk,ss,d}$：日付$d$における1日当たりのソーラーシステムのタンク蓄熱量の上限による補正集熱量(MJ/d) / Corrected heat collection by the upper limit of the tank thermal storage capacity of the solar system per day at date $ d $  
$HC_{tnk,ss,d}$：日付$d$におけるソーラーシステムのタンク蓄熱量の上限(MJ) / The upper limit of the tank heat storage capacity of the solar system on the date $ d $  
$Q_{ss,d}$：日付$d$における1日当たりの基準集熱量(MJ/d) / Standard heat collection per day for date $ d $  
$\alpha_{tnk,ss,d}$：日付$d$におけるソーラーシステムのタンク有効利用率(1/d) / Tank utilization rate of solar system at date $ d $  


In [ ]:
def calc_L_tnk_ss_d(Q_ss_d, W_tnk_ss, Theta_wtr_d):
    """1日当たりのソーラーシステムのタンク蓄熱量の上限による補正集熱量 (11)

    :param Q_ss_d: 1日当たりの基準集熱量 (MJ/d)
    :type Q_ss_d: ndarray
    :param W_tnk_ss: ソーラーシステムのタンク容量 (L)
    :type W_tnk_ss: float
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :type Theta_wtr_d: ndarray
    :return: 1日当たりのソーラーシステムのタンク蓄熱量の上限による補正集熱量 (MJ/d)
    :rtype: ndarray
    """
    # ソーラーシステムのタンク蓄熱量 (MJ)
    HC_tnk_ss_d = get_HC_tnk_ss_d(W_tnk_ss, Theta_wtr_d)

    # ソーラーシステムのタンク有効利用率 (-)
    alpha_tnk_ss_d = get_alpha_tnk_ss_d()

    # 1日当たりのソーラーシステムのタンク蓄熱量の上限による補正集熱量 (9)
    # Q_ss_d および HC_tnk_ss_d * alpha_tnk_ss_d の1日ごとの最小をとるために、axis=0を指定
    L_tnk_ss_d = np.min([Q_ss_d, HC_tnk_ss_d * alpha_tnk_ss_d], axis=0)

    return L_tnk_ss_d


日付$d$におけるソーラーシステムのタンク有効利用率$\alpha_{tnk,ss,d}$は、日付$d$によらず1.0に等しい。

The tank effective utilization rate $\alpha_{tnk, ss, d} $ of the solar system at the date $ d $ is equal to 1.0 regardless of the date $ d $.

In [ ]:
def get_alpha_tnk_ss_d():
    """ソーラーシステムのタンク有効利用率 (-)

    :return: ソーラーシステムのタンク有効利用率 (-)
    :rtype: float
    """
    return 1.0


$$\large
HC_{tnk,ss,d} = (\theta_{tnk,ss} - \theta_{wtr,d}) \times W_{tnk,ss} \times 4.186 \times 10^{-3}
$$

$HC_{tnk,ss,d}$：日付$d$におけるソーラーシステムのタンク蓄熱量の上限(MJ) / The upper limit of the tank heat storage capacity of the solar system on the date $ d $  
$W_{tnk,ss}$：ソーラーシステムのタンク容量(L) / Solar system tank capacity  
$\theta_{tnk,ss}$：ソーラーシステムのタンク内温度(℃) / Tank temperature in the solar system  
$\theta_{wtr,d}$：日付$d$における日平均給水温度(℃) / Average daily water supply temperature on date $ d $  

In [ ]:
def get_HC_tnk_ss_d(W_tnk_ss, Theta_wtr_d):
    """ソーラーシステムのタンク蓄熱量 (12)

    :param W_tnk_ss: ソーラーシステムのタンク容量 (L)
    :type W_tnk_ss: float
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :type Theta_wtr_d: ndarray
    :return: ソーラーシステムのタンク蓄熱量 (MJ)
    :rtype: ndarray
    """
    # ソーラーシステムのタンク内温度 (℃)
    Theta_tnk_ss = get_Theta_tnk_ss()

    # ソーラーシステムのタンク蓄熱量 (10)
    HC_tnk_ss_d = (Theta_tnk_ss - Theta_wtr_d) * W_tnk_ss * 4.186 * 10 ** (-3)

    return HC_tnk_ss_d


ソーラーシステムのタンク内温度$\theta_{tnk,ss}$は65℃とする。 

The in-tank temperature $ \theta_{tnk, ss} $ of the solar system is 65 °C.

In [ ]:
def get_Theta_tnk_ss():
    """ソーラーシステムのタンク内温度 (℃)

    :return: ソーラーシステムのタンク内温度 (℃)
    :rtype: float
    """
    return 65.0


#### 7.2 基準集熱量 / Reference heat collection

$$\large
Q_{ss,d} = Q_{sp,ss,d} \times A_{sp,ss} \times f_{sp,ss} \times f_{s,ss}
$$

$A_{sp,ss}$：太陽熱集熱部の有効集熱面積(m<sup>2</sup>) / Effective heat collection surface of solar heat collector  
$Q_{ss,d}$：日付$d$における1日当たりの基準集熱量(MJ/d) / Standard heat collection per day for date $ d $  
$Q_{sp,ss,d}$：日付$d$における1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量(MJ/(m<sup>2</sup>・d)) / The amount of solar radiation per unit area of the solar thermal collector installation surface per day for the date $ d $  
$f_{s,ss}$：液体集熱式太陽熱利用設備のシステム効率(-) / System efficiency of liquid collection solar system  
$f_{sp,ss}$：太陽熱集熱部の集熱効率(-) / Heat collection efficiency of solar heat collector  

In [ ]:
def calc_Q_ss_d(A_sp_ss, P_alpha_sp_ss, P_beta_sp_ss, region, sol_region):
    """1日当たりの基準集熱量 (13)

    :param A_sp_ss: ソーラーシステムの太陽熱集熱部の有効集熱面積 (m2)
    :type A_sp_ss: float
    :param P_alpha_sp_ss: ソーラーシステムの太陽熱集熱部の方位角 (°)
    :type P_alpha_sp_ss: float
    :param P_beta_sp_ss: ソーラーシステムの太陽熱集熱部の傾斜角 (°)
    :type P_beta_sp_ss: float
    :param region:　省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 1日当たりの基準集熱量 (MJ/d))
    :rtype: ndarray
    """
    # 1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量
    Q_sp_ss_d = calc_Q_sp_ss_d(P_alpha_sp_ss, P_beta_sp_ss, region, sol_region)

    # 太陽熱集熱部の集熱効率
    f_sp_ss = get_f_sp_ss()

    # 液体集熱式太陽熱利用設備のシステム効率
    f_s_ss = get_f_s_ss()

    # 1日当たりの基準集熱量 (11)
    Q_ss_d = Q_sp_ss_d * A_sp_ss * f_sp_ss * f_s_ss

    return Q_ss_d


太陽熱集熱部の集熱効率$f_{sp,ss}$は、0.4とする。液体集熱式太陽熱利用設備のシステム効率$f_{s,ss}$は、0.85とする。 

The heat collection efficiency $ f_ {sp, ss} $ of the solar heat collector is 0.4. The system efficiency $ f_ {s, ss} $ of the liquid collection solar system is 0.85.

In [ ]:
def get_f_sp_ss():
    """太陽熱集熱部の集熱効率 (-)

    :return: 太陽熱集熱部の集熱効率 (-)
    :rtype: float
    """
    return 0.4


def get_f_s_ss():
    """液体集熱式太陽熱利用設備のシステム効率 (-)

    :return: 液体集熱式太陽熱利用設備のシステム効率 (-)
    :rtype: float
    """
    return 0.85


太陽熱集熱部の有効集熱面積$A_{sp,ss}$は、ISO 9488 の 8.6 節「aperture area」に規定される方法により算出した値とするか、JIS A 4112 に規定される集熱器総面積に 0.85 を乗じた値とする。 

The effective heat collecting area $ A_ {sp, ss} $ of the solar heat collecting part is a value calculated by the method defined in Section 8.6 “aperture area” of ISO 9488, or the heat collector specified in JIS A 4112. The total area of the equipment is multiplied by 0.85.

In [ ]:
def get_A_sp_ss_JIS_A_4112(A):
    """太陽熱集熱部の有効集熱面積 (m2)

    :param A: 太陽熱集熱部総面積 (m2)
    :type A: float
    :return: 太陽熱集熱部の有効集熱面積 (m2)
    :rtype: float
    """
    return A * 0.85


$$\large
Q_{sp,ss,d} = \sum_{t = 0}^{23} ( I_{s,sp,ss,d,t} \times 3600 \times 10^{-6} )
$$

$I_{s,sp,ss,d,t}$：日付$d$の時刻$t$における太陽熱集熱部設置面の単位面積当たりの平均日射量(W/m<sup>2</sup>) / Average solar radiation amount per unit area of solar heat collector installation surface at time $ t $ of date $ d $  
$Q_{sp,ss,d}$：日付$d$における1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量(MJ/m<sup>2</sup>・d) / Average solar radiation amount per unit area of solar heat collector installation surface at time $ t $ of date $ d $  

In [ ]:
def calc_Q_sp_ss_d(P_alpha_sp_ss, P_beta_sp_ss, region, sol_region):
    """1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量 (MJ/(m2・d)) (14)

    :param P_alpha_sp_ss: ソーラーシステムの太陽熱集熱部の方位角 (°)
    :type P_alpha_sp_ss: float
    :param P_beta_sp_ss: ソーラーシステムの太陽熱集熱部の傾斜角 (°)
    :type P_beta_sp_ss: float
    :param region:　省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量 (MJ/(m2・d))
    :rtype: ndarray
    """
    # 太陽熱集熱部設置面の単位面積当たりの平均日射量
    solrad = load_solrad(region, sol_region)
    I_s_sp_ss_d_t = calc_I_s_d_t(P_alpha_sp_ss, P_beta_sp_ss, solrad)

    # 1日当たりの太陽熱集熱部設置面の単位面積当たりの日射量を求める
    # 8760時間が一次配列で与えられるので、365*24の2次配列へ並べ替える
    tmp = I_s_sp_ss_d_t.reshape(365, 24)

    # 配列の2次の軸(=各日24時間分)を加算する
    tmp = np.sum(tmp, axis=1)

    # 単位をWからMJへ換算する
    Q_sp_ss_d = tmp * 3600 * 10 ** (-6)

    return Q_sp_ss_d


#### 7.3 循環ポンプの消費電力量 / Power consumption of circulation pump

$$\large
E_{E,ss,cp,d,t} = P_{cp,ss} \times t_{cp,ss,d,t} \times 10^{-3}
$$

$E_{E,ss,cp,d,t}$：日付$d$の時刻$t$における1時間当たりのソーラーシステムの循環ポンプの消費電力量(kWh/h) / Solar system circulation pump power consumption per hour at time $ t $ on date $ d $  
$P_{cp,ss}$：ソーラーシステムの循環ポンプの消費電力(W) / Solar system circulation pump power consumption  
$t_{cp,ss,d,t}$：日付$d$の時刻$t$における1時間当たりのソーラーシステムの循環ポンプの稼働時間(h/h) / Solar system circulation pump uptime per hour at time $ t $ on date $ d $  

In [ ]:
def calc_E_E_ss_cp_d_t(pmp_type, P_alpha_sp_ss, P_beta_sp_ss, region, sol_region):
    """1時間当たりのソーラーシステムの循環ポンプの消費電力量 (15)

    :param pmp_type: ソーラーシステムの循環ポンプの種類 (省消費電力型,上記以外の機種)
    :type pmp_type: str
    :param P_alpha_sp_ss: ソーラーシステムの太陽熱集熱部の方位角 (°)
    :type P_alpha_sp_ss: float
    :param P_beta_sp_ss: ソーラーシステムの太陽熱集熱部の傾斜角 (°)
    :type P_beta_sp_ss: float
    :param region:　省エネルギー地域区分
    :type region: int
    :param sol_region: 年間日射地域区分
    :type sol_region: int
    :return: 1時間当たりのソーラーシステムの循環ポンプの消費電力量 (kWh/h)
    :rtype: ndarray
    """
    # ソーラーシステムの循環ポンプの消費電力 (W)
    P_cp_ss = get_P_cp_ss(pmp_type)

    # 太陽熱集熱部設置面の単位面積当たりの平均日射量 (W/m2)
    solrad = load_solrad(region, sol_region)
    I_s_sp_ss_d_t = calc_I_s_d_t(P_alpha_sp_ss, P_beta_sp_ss, solrad)

    # 1時間当たりのソーラーシステムの循環ポンプの稼働時間
    t_cp_ss_d = get_t_cp_ss_d_t(I_s_sp_ss_d_t)

    return P_cp_ss * t_cp_ss_d * 10 ** (-3)


<div style="text-align: center;font-weight: bold;">表　ソーラーシステムの循環ポンプの消費電力$P_{cp,ss}$の値と適用条件<br>Table: Solar system circulation pump power consumption $ P_ {cp, ss} $ values and application conditions</div>

|項目<br>item|適用条件<br>Applicable condition|循環ポンプの消費電力$P_{cp,ss}$<br>Power consumption of circulation pump|
|-|-|-|
|省消費電力型<br>Power saving type|現時点では該当するポンプがない<br>There is no corresponding pump at present|40|
|上記以外の機種<br>Other models than above|すべてのポンプ<br>All pumps|80|

In [ ]:
def get_P_cp_ss(pmp_type):
    """ソーラーシステムの循環ポンプの消費電力

    :param pmp_type: ソーラーシステムの循環ポンプの種類 (省消費電力型,上記以外の機種)
    :type pmp_type: str
    :return: ソーラーシステムの循環ポンプの消費電力 (W)
    :rtype: float
    """
    # 表3 ソーラーシステムの循環ポンプの消費電力の値と適用条件
    table_3 = (40.0, 80.0)

    if pmp_type == '省消費電力型':
        return table_3[0]
    elif pmp_type == '上記以外の機種':
        return table_3[1]
    else:
        raise ValueError(pmp_type)


#### 7.4 循環ポンプの稼働時間 / Circulating pump operating time

$I_{s,lwlim,cp,ss} \leq I_{s,sp,ss,d,t}$の場合： 

In the case of $I_{s,lwlim,cp,ss} \leq I_{s,sp,ss,d,t}$

$$\large
t_{cp,ss,d,t} = 1
$$

$I_{s,sp,ss,d,t} < I_{s,lwlim,cp,ss}$の場合： 

In the case of $I_{s,sp,ss,d,t} < I_{s,lwlim,cp,ss}$

$$\large
t_{cp,ss,d,t} = 0
$$

$I_{s,lwlim,cp,ss}$：循環ポンプが稼働する太陽熱集熱部設置面の単位面積当たりの日射量の下限(W/m<sup>2</sup>) / Lower limit of the amount of solar radiation per unit area of the solar heat collector installation surface where the circulation pump operates  
$I_{s,sp,ss,d,t}$：日付$d$の時刻$t$における太陽熱集熱部設置面の単位面積当たりの平均日射量(W/m<sup>2</sup>) / Average solar radiation amount per unit area of solar heat collector installation surface at time $ t $ of date $ d $  
$t_{cp,ss,d,t}$：日付$d$の時刻$t$における1時間当たりのソーラーシステムの循環ポンプの稼働時間(h/h) / Solar system circulation pump uptime per hour at time $ t $ on date $ d $  

In [ ]:
def get_t_cp_ss_d_t(I_s_sp_ss_d_t):
    """1時間当たりのソーラーシステムの循環ポンプの稼働時間 (h/h) (16)

    :param I_s_sp_ss_d_t: 傾斜面の単位面積当たりの平均日射量 (W/m2)
    :type I_s_sp_ss_d_t: ndarray
    :return: 1時間当たりのソーラーシステムの循環ポンプの稼働時間 (h/h)
    :rtype: ndarray
    """
    # ソーラーシステムの循環ポンプが稼働する太陽熱集熱部設置面の単位面積当たりの日射量の下限
    I_s_lwlim_cp_ss = get_I_s_lwlim_cp_ss()

    # 1時間当たりのソーラーシステムの循環ポンプの稼働時間の計算領域を確保
    t_cp_ss_d_t = np.zeros(24 * 365, dtype=np.int32)

    # I_s_lwlim_cp_ss <= I_s_sp_ss_d_t の場合
    f1 = I_s_lwlim_cp_ss <= I_s_sp_ss_d_t
    t_cp_ss_d_t[f1] = 1  # (15-1)

    # I_s_sp_ss_d_t < I_s_lwlim_cp_ss  の場合
    f2 = I_s_sp_ss_d_t < I_s_lwlim_cp_ss
    t_cp_ss_d_t[f2] = 0  # (15-2)

    return t_cp_ss_d_t


ソーラーシステムの循環ポンプが稼働する太陽熱集熱部設置面の単位面積当たりの日射量の下限$I_{s,lwlim,cp,ss}$は、150 W/m<sup>2</sup> とする。 

The lower limit $ 1_ {s, lwlim, cp, ss} $ of the amount of solar radiation per unit area of the solar heat collecting part installation surface on which the circulation pump of the solar system operates is 150 W/m<sup>2</sup>.

In [ ]:
def get_I_s_lwlim_cp_ss():
    """ソーラーシステムの循環ポンプが稼働する太陽熱集熱部設置面の単位面積当たりの日射量の下限 (W/2m)

    :return: ソーラーシステムの循環ポンプが稼働する太陽熱集熱部設置面の単位面積当たりの日射量の下限 (W/2m)
    :rtype: float
    """
    return 150.0


## 付録A / Appendix A 

省略

Omission